In [ ]:
import tarfile
import os.path
import json
import re
from bz2 import BZ2File
from urllib import request
from io import BytesIO

import numpy as np

fname = "cmv.tar.bz2"
f = open(fname, 'rb')


tar = tarfile.open(fileobj=f, mode="r")

# Extract the file we are interested in

train_fname = "all/train_period_data.jsonlist.bz2"

train_bzlist = tar.extractfile(train_fname)

# Deserialize the JSON list
data_train = [
    json.loads(line.decode('utf-8'))
    for line in BZ2File(train_bzlist)
]


In [ ]:
data1=[]
for item in data_train[0:20]:
    data1.append(item)


In [ ]:
import array
import re
from nltk import word_tokenize, pos_tag
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_features = 40000
tfidf_vectorizer = TfidfVectorizer(max_features=no_features)

#Function for tfidf_interact
def word_over(strC, strO): 
    a = set(strC.split()) 
    b = set(strO.split())
    c = a.intersection(b)
    f1 = float(len(c))
    f2 = float(len(c))/(len(a))
    f3 = float(len(c))/(len(b))
    f4 = float(len(c))/(len(a) + len(b) - len(c))
    return [f1,f2,f3,f4]

#Function for clean up using regex
def remreg(cmv_post):
    xy=re.sub('[^a-zA-Z0-9][(/w)]', '', cmv_post)
    return xy

#Function for clean up
def cleanup(cmv_post):
    lines = [line for line in cmv_post.splitlines()
             if not line.lstrip().startswith("&gt;")
             and not line.lstrip().startswith("____")
             and "edit" not in " ".join(line.lower().split()[:2])
            ]
    return "\n".join(lines)

inpdat=[]
y=0
#Loop that gives each discussion
for idx,node in enumerate(data1):
    t1=tuple()
    op_node=dict()
    comments=[]
    word4=[]
    #Cleaning the opinion post
    if data1[idx]['selftext'] == None:
        del data1[idx]
        break
    a = cleanup(data1[idx]['selftext'])
    word2 = ""
    #print(idx)
    for word in a.split(' '):
        word1 = remreg(word)
        word2 = word2+word1+" "
    for w1 in word2.split():
        word4.append(w1)
    if len(word4)<100:
        del data1[idx]
        break
    op_node["parent_id"]=data1[idx]['subreddit_id']
    for sent in word2.split('.'):
        postname=sent
        break
    op_node["post_name"]=postname
    op_node["text"]=word2
    
    comments = []
    word6 = ""
    #node_tfidf_features = []
    x=0
    #loop that gives each comment of the discussion
    for item in data1[idx]['comments']:
        comment=dict()
        l1=[]
        if 'body' in item.keys():
            c1 = cleanup(item['body'])
            for word in c1.split(' '):
                word5 = remreg(word)
                word6 = word6+word5+" "
            if word6 == " ":
                break
            if word6 == "[deleted] ":
                break
            l1.append(word6)
            tfidf = tfidf_vectorizer.fit_transform(l1)
            comment["node_tfidf"]=tfidf
            #tfidf_feature_names = [tfidf_vectorizer.get_feature_names()]
            #node_tfidf_features.append(tfidf_feature_names)
            comment["tfidf_interact"]=word_over(word6,word2)
            if item['author_flair_text']:
                comment["delta"]=1
            else: comment["delta"]=0
            comment["text"]=word6
            comments.append(comment)
            word6=""
            word5=""
            x=x+1
    t1=(op_node,comments) #Main tuple for each discussion
    inpdat.append(t1)     #inpdat is supposed to be the json pickled file for input
y=y+1
